In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from refuge.config import load_config
from refuge.training import train, get_tokenizer_and_model

In [ ]:
cfg = load_config()
cfg

In [ ]:
tokenizer, model = get_tokenizer_and_model(cfg)

In [ ]:
nearest_tokens_for_soft_prompt = model.translated_soft_prompt()
tokenizer.decode(nearest_tokens_for_soft_prompt)

In [ ]:
len(nearest_tokens_for_soft_prompt)

In [ ]:
# eos_token_id = tokenizer.encode("### End")[0]

In [ ]:
import random

In [ ]:
10**0

In [ ]:
1e4

In [ ]:
a = random.randint(1, 99)
b = random.randint(1, 99)
c = a + b

prompt = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n" + 
    model.soft_prompt + 
    f"\n{a} + {b}\n\n"
    "### Response:\n" # + str(c) + "\n\n### End"
)

print(prompt)

In [ ]:
eos_token_id = tokenizer.encode("### End")[0]

In [ ]:
call = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

output = model.generate(
    input_ids=call,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=128,
    # top_p=0.92,
    # do_sample=True,
    eos_token_id=eos_token_id,
)

print(tokenizer.decode(output[0]))

In [ ]:
c

In [ ]:
# 6191

In [ ]:
train(cfg, tokenizer, model)

In [ ]:
def test_adder(tokenizer, model, prompt_template, soft_prompt, a, b):
    prompt = prompt_template.format(soft_prompt=soft_prompt, a=a, b=b)
    
    call = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    
    response_string = "### Response:"
    end_string = "### End"
    eos_token_id = tokenizer.encode(end_string)[0]
    
    output = model.generate(
        input_ids=call,
        pad_token_id=tokenizer.pad_token_id,
        max_new_tokens=128,
        eos_token_id=eos_token_id,
    )
    
    result_string = tokenizer.decode(output[0])
    
    value = result_string.split(response_string)[-1].split(end_string)[0].strip()
    
    try:
        return int(''.join(filter(str.isdigit, value)))
    except:
        return None

In [ ]:
prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{soft_prompt}
{a} + {b}

### Response:"""

In [ ]:
def test_accuracy(num_digits, soft_prompt, n=100):
    correct = 0
    
    for _ in range(n):
        a = random.randint(10 ** (num_digits - 1), 10**num_digits - 1)
        b = random.randint(10 ** (num_digits - 1), 10**num_digits - 1)
        c = a + b

        if c == test_adder(tokenizer, model, prompt_template, soft_prompt, a, b):
            correct += 1
            
    return correct / n

In [ ]:
num_digits = 3
soft_prompt = ""

accuracy_without_soft_prompt = test_accuracy(num_digits, soft_prompt)
accuracy_without_soft_prompt

In [ ]:
accuracy_with_soft_prompt = test_accuracy(num_digits, model.soft_prompt)
accuracy_with_soft_prompt